In [1]:
import re
import pandas as pd
import datasets

c:\Users\alinz\.conda\envs\RAG11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
data = pd.read_csv('RAG_with_LLm/data/dialogues.tsv', sep='\t')
data[:2]

,persona_1_profile,persona_2_profile,dialogue
0,<span class=participant_1>У меня любимая работ...,<span class=participant_2>Ищу принца.<br />Вед...,<span class=participant_2>Пользователь 2: Прив...
1,<span class=participant_1>Я работаю учителем<b...,<span class=participant_2>Я бизнесмен<br />У м...,<span class=participant_1>Пользователь 1: Прив...


In [5]:
def replace_spans(text):
    text = re.sub(r'<span class=participant_2>Пользователь 2:', "<p-2>:", text)
    text = re.sub(r'<span class=participant_1>Пользователь 1:', "<p-1>:", text)
    text = re.sub(r'</span>', " ", text)
    text = re.sub(r'<br />\)', " ", text)
    return text

rows = []
for dialogue in data["dialogue"]:
    dialogue = replace_spans(dialogue)
    turns = dialogue.split('<br />')
    turns = [turn.strip() for turn in turns if turn.strip()]
    history = []
    for i in range(len(turns) - 1):
        current_turn = turns[i].split(':', 1)
        next_turn = turns[i+1].split(':', 1)
        if len(current_turn) < 2 or len(next_turn) < 2:
            continue
        if '<p-2>' in current_turn[0]:
            history.append({'speaker': '<p-2>', 'text': current_turn[1].strip()})
        else:
            history.append({'speaker': '<p-1>', 'text': current_turn[1].strip()})
        query = history.copy()
        if '<p-2>' in next_turn[0]:
            candidate = {'speaker': '<p-2>', 'text': next_turn[1].strip()}
        else:
            candidate = {'speaker': '<p-1>', 'text': next_turn[1].strip()}
        rows.append({'history': query, 'turn': candidate})

df = pd.DataFrame(rows)


def combine_texts_with_tokens(texts):
    return " ".join([f"{text['speaker']} {text['text']}" for text in texts])

df['query'] = df['history'].apply(combine_texts_with_tokens)
df['candidate'] = df['turn'].apply(lambda x: f"{x['speaker']} {x['text']}")
df = df[['query', 'candidate']]
df

,query,candidate
0,<p-2> Привет) расскажи о себе,<p-1> Привет) под вкусный кофеек настроение по...
1,<p-2> Привет) расскажи о себе <p-1> Привет) по...,<p-2> Что читаешь? Мне нравится классика
2,<p-2> Привет) расскажи о себе <p-1> Привет) по...,<p-2> Я тоже люблю пообщаться
3,<p-2> Привет) расскажи о себе <p-1> Привет) по...,"<p-1> Люблю животных, просто обожаю, как и сво..."
4,<p-2> Привет) расскажи о себе <p-1> Привет) по...,<p-1> Я фантастику люблю
...,...,...
211292,"<p-2> Привет! рада новому знакомству, как твои...",<p-1> Понятно я из школы только програматику м...
211293,"<p-2> Привет! рада новому знакомству, как твои...",<p-2> а я наоборот не любила. Для меня ближе к...
211294,"<p-2> Привет! рада новому знакомству, как твои...",<p-2> Жизнь хорошо) вот тока пришла с бассейна.
211295,"<p-2> Привет! рада новому знакомству, как твои...",<p-2> Хоть он и находится далеко всегда возвра...


In [8]:
train_split = df[:209297 ]
val_split = df[209297:210297]
test_split = df[210297:]

In [9]:
train = datasets.Dataset.from_pandas(train_split)
val = datasets.Dataset.from_pandas(val_split)
test = datasets.Dataset.from_pandas(test_split)
data = datasets.DatasetDict({'train': train, 'val': val, 'test': test})
data

DatasetDict({
    train: Dataset({
        features: ['query', 'candidate'],
        num_rows: 209297
    })
    val: Dataset({
        features: ['query', 'candidate'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['query', 'candidate'],
        num_rows: 1000
    })
})

In [10]:
data.save_to_disk('RAG_with_LLm/data/toloka_data')

Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 199881.05 examples/s]
